In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [17]:
# 한 건의 대화 정보를 저장하는 클래스
class Conversation:
    # 생성자 함수 => __init__(self)
    # 클래스 내부에서 정의하는 함수는 무조건 첫 번째 인수로 자기 자신의 객체가 메모리에 생성된 주소를 넘겨받는 self라는 인수를 써야한다.
    # 클래스의 객체가 생성될 때 자동으로 실행되는 함수로 클래스의 멤버 변수(self가 앞에 붙은 변수)를 초기화 시킨다.
    def __init__(self, question, answer):
        # print("Conversation 클래스의 생성자 함수가 실행됩니다.")
        # 멤버 변수는 자기 자신의 객체가 생성된 메모리의 주소를 저장하는 인수인 self를 사용해서 선언한다.
        # self를 앞에 붙여준 변수를 멤버 변수라 하고 클래스 내부의 모든 함수에서 사용할 수 있다. self를 앞에 붙여주지 않은 함수는 지역 변수라 하고
        # 변수가 선언된 함수 내부에서만 사용할 수 있다.
        self.question = question # 질문을 기억하는 멤버 변수를 선언하고 초기화 시킨다.
        self.answer = answer     # 답변을 기억하는 멤버 변수를 선언하고 초기화 시킨다.
    
    # 클래스에 저장된 데이터를 출력하는 함수 => __str__(self)
    # __str__(self) 함수를 정의하지 않으면 클래스의 객체가 메모리에 생성된 주소가 나온다.
    # 클래스에 저장된 데이터를 출력할 형태로 편집해서 return 시킨다.
    def __str__(self):
        # print(self.question)
        return "question : " + self.question + "\nanswer : " + self.answer

In [18]:
# 클래스의 객체를 생성하는 방법
# 변수(객체)이름 = 클래스이름()
c = Conversation("질문", "답변")
print(c)

question : 질문
answer : 답변


In [36]:
# 75개의 대화 주제와 세부 대화 내용의 url를 크롤링해서 리스트에 저장한 후 리턴하는 함수
def getSubject():
    subjects = []     # 대화 주제를 기억할 빈 리스트
    contentLinks = [] # 대화 주제에 따른 세부 대화 내용의 url을 기억할 빈 리스트
    request = requests.get("https://basicenglishspeaking.com/daily-english-conversation-topics/")
    html = request.text
    soup = BeautifulSoup(html, "html.parser")
    divs = soup.findAll("div", {"class": "tcb-flex-col"})
    # print(divs)
    for div in divs:
        # print(div)
        chapters = div.findAll("a")
        for chapter in chapters:
            subjects.append(chapter.text) # subjects 리스트에 크롤링한 대화 주제를 저장한다.
            # get("href") : 인수로 지정된 속성의 속성값만 얻어온다.
            # print(chapter.get("href"))
            contentLinks.append(chapter.get("href"))
        # ===== for chapter in chapters: 끝
    # ===== for div in divs: 끝
    
    # 대화 주제와 세부 대화 내용의 url이 저장된 리스트를 리턴시킨다.
    return subjects, contentLinks # 2건의 데이터를 리턴하는 것이 아니고 2건의 데이터로 구성된 튜플 1개를 리턴시킨다.

In [58]:
if __name__ == "__main__":
    subjects, contentLinks = getSubject()
    # for i in range(len(subjects)):
        # print("{0:2d}. {1} - {2}".format(i + 1, subjects[i], contentLinks[i]))
    print("총 {}개의 대화를 찾았습니다.".format(len(subjects)))
    
    # 대화 주제에 따른 모든 대화 내용을 저장할 빈 리스트를 선언한다.
    conversation = [] # Conversation 클래스의 객체가 저장된다. => 질문과 답변이 한 쌍으로 저장된 클래스
    
    i = 0 # 대화 주제의 개수를 카운트 하는 변수
    # 대화 주제의 개수 만큼 반복하며 대화 내용을 얻어온다. => 테스트는 전체를 대상으로 하면 시간이 많이 걸리므로 슬라이싱 기능을 이용해 일부만 대상으로 한다.
    for subject in subjects[:3]:
        # print("{0:2d}. {1} - {2}".format(i + 1, subjects[i], contentLinks[i]))
        conversation.append("{0:2d}. {1} - {2}".format(i + 1, subjects[i], contentLinks[i]))
        
        # 대화 주제별 대화 내용을 크롤링할 페이지를 서버에 요청한다.
        request = requests.get(contentLinks[i])
        
        # time 모듈을 import 하면 sleep() 메소드를 사용할 수 있다.
        # sleep() 메소드는 인수로 지정된 시간만큼 프로그램을 멈춘다. => 시간을 초 단위로 지정한다.
        # 대형 포털 사이트는 짧은 간격으로 많은 요청이 들어오면 자기네 사이트가 공격당하는 것으로 판단할 수 있기 때문에 일정한 시간 간격을 두고
        # 사이트의 정보를 요청한다.
        # 페이지가 로딩되는 시간만큼 기다리기 위해 일정한 시간 간격을 두고 정보를 요청한다.
        time.sleep(1)
        
        html = request.text
        soup = BeautifulSoup(html, "html.parser")
        
        # 대화 주제에 따른 대화 내용은 class 속성이 "sc_player_container1"인 div 태그의 바로 다음에 위치한 형제이다.
        divs = soup.findAll("div", {"class": "sc_player_container1"})
        # print(len(divs)) # 대화 내용의 개수
        
        # 각각의 대화 주제에 따른 대화 내용의 개수 만큼 반복한다.
        for div in divs:
            # index() : 인수로 지정된 객체의 인덱스 번호를 얻어온다.
            # divs.index(div) => 대화 내용 전체(divs)에서 특정 대화(div)의 인덱스(인련) 번호를 얻어온다.
            # index() 메소드의 실행 결과가 짝수면 질문이고 홀수면 답변이다.
            # print(divs.index(div))
            
            # 크롤링할 데이터가 <div> 태그 내부에 작성된 것이 아니고 class 속성이 "sc_player_container1"인 <div> 태그 다음 형제로 되어있다.
            # next_sibling => 다음 형제 노드를 의미한다.
            # previous__sibling => 이전 형제 노드를 의미한다.
            if divs.index(div) % 2 == 0: # 짝수인가? => 질문인가?
                question = div.next_sibling # 인덱스가 짝수면 질문 변수에 질문을 저장한다.
                # print("질문 : {}".format(question))
            else: 
                answer = div.next_sibling # 인덱스가 홀수면 답변 변수에 답변을 저장한다.
                # print("답변 : {}".format(answer))
                
                # 질문과 답변이 한 쌍이 되었으면 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
                c = Conversation(question, answer)
                # print(c)
                # 질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억할 conversation 리스트에 저장한다.
                conversation.append(c)
                
        # 대화 주제 1건이 처리 완료되었으므로 다음 대화를 처리하기 위해서 i값을 1증가 시킨다.
        i += 1
    # ===== for subject in subjects: 끝
                            
    for c in conversation:
        print(c)

총 75개의 대화를 찾았습니다.
 1. Family - https://basicenglishspeaking.com/family/
question : How many people are there in your family?
answer : There are 5 people in my family: my father, mother, brother, sister, and me.
question : Does your family live in a house or an apartment?
answer : We live in a house in the countryside.
question : What does your father do?
answer : My father is a doctor. He works at the local hospital.
question : How old is your mother?
answer : She is 40 years old, 1 year younger than my father.
question : Do you have any siblings? What’s his/her name?
answer : Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
question : Are you the oldest amongst your brothers and sisters?
answer : No, I’m not. I’m the second child in my family.
question : What do your mother/father like?
answer : My father likes playing football and my mother likes cooking.
question : Do your parents let you stay out late?
answer : Of course not. They always ask me to get home befo